###### Imports and Settings

In [1]:
GNRC = ['111', '161', '125', '083', '085', '043', '021', '147', '165', '037', '189', '169', '187', '149', '119']
KY = ['003', #Allen
      '213'] #Simpson
slaces = ['02180', #Ashland City: Cheatham
          '39660', #Kingston Springs: Cheatham
          '57480', #Pegram: Cheatham
          '59560', #Pleasant View: Cheatham
          '04620', #Belle Meade: Davidson
          '05140', #Berry Hill: Davidson
          '27020', #Forest Hills: Davidson
          '29920', #Goodlettsville: Davidson/Sumner
          '52006', #Nashville-Davidson metropolitan government (balance): Davidson
          '54780', #Oak Hill: Davidson
          '63140', #Ridgetop: Davidson/Robertson
          '09880', #Burns: Dickson
          '13080', #Charlotte: Dickson
          '20620', #Dickson: Dickson
          '69080', #Slayden: Dickson
          '76860', #Vanleer: Dickson
          '79980', #White Bluff: Dickson
          '24320', #Erin: Houston
          '73460', #Tennessee Ridge: Houston/Stewart
          '44840', #McEwen: Humphreys
          '52820', #New Johnsonville: Humphreys
          '78560', #Waverly: Humphreys
          '16540', #Columbia: Maury
          '51080', #Mount Pleasant: Maury
          '70580', #Spring Hill: Maury/Williamson
          '15160', #Clarksville: Montgomery
          '00200', #Adams: Robertson
          '11980', #Cedar Hill: Robertson
          '16980', #Coopertown: Robertson
          '18420', #Cross Plains: Robertson
          '30960', #Greenbrier: Robertson
          '48980', #Millersville: Robertson/Sumner
          '60280', #Portland: Robertson/Sumner
          '70500', #Springfield: Robertson
          '80200', #White House: Robertson/Sumner
          '22360', #Eagleville: Rutherford
          '41200', #La Vergne: Rutherford
          '51560', #Murfreesboro: Rutherford
          '69420', #Smyrna: Rutherford
          '18820', #Cumberland City: Stewart
          '21400', #Dover: Stewart
          '28540', #Gallatin: Sumner
          '33280', #Hendersonville: Sumner
          '79420', #Westmoreland: Sumner
          '08280', #Brentwood: Williamson
          '25440', #Fairview: Williamson
          '27740', #Franklin: Williamson
          '53460', #Nolensville: Williamson
          '73900', #Thompson's Station: Williamson
          '41520', #Lebanon: Wilson
          '50780', #Mount Juliet: Wilson
          '78320', #Watertown: Wilson
          '40160', #Lafayette: Macon
          '69114', #Scottsville city, Kentucky
          '28918', #Franklin city, Kentucky
          '49460'] #Mitchelville city: Sumner

In [2]:
import pandas as pd
import numpy as np
import requests
from collections import deque
from functools import reduce
import pickle
import matplotlib.pyplot as plt
#pd.options.display.float_format = '{:,.0f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y

In [4]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)
#create a variable that contains your api key
census_key = keys_dict_2['CENSUS']
bea_key = keys_dict_2['BEA']
bls_key = keys_dict_2['BLS']
api_key = keys_dict_2['CENSUS']

# Get 2020 PL population #

In [5]:
#2020 PL for households and population #s
#counties
url_str= 'https://api.census.gov/data/2020/dec/pl?key='+api_key
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'P1_001N', 'H1_001N']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Population', 'Households', 'StateFIPS', 'GeoFIPS']
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)]
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'P1_001N', 'H1_001N']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21" 
data = requests.get(url_str, params= predicates)                                                              
col_names = ['NAME', 'GEO_ID', 'Population', 'Households', 'StateFIPS', 'GeoFIPS']
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)]
df = pd.concat([df, kycos], axis = 0)
#ky places call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'P1_001N', 'H1_001N']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Population', 'Households', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GeoFIPS'].isin(slaces)]
df = pd.concat([df, places], axis = 0)
#places
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'P1_001N', 'H1_001N']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47" 
data = requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Population', 'Households', 'StateFIPS', 'GeoFIPS']
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GeoFIPS'].isin(slaces)]
df = pd.concat([df, places], axis = 0)
#state call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'P1_001N', 'H1_001N']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Population', 'Households', 'StateFIPS']
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0'
df = pd.concat([df, state], axis = 0)
#national call
predicates= {}
get_vars= ["NAME", 'GEO_ID', 'P1_001N', 'H1_001N']
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = ['NAME', 'GEO_ID', 'Population', 'Households', 'StateFIPS']
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0'
df = pd.concat([df, national], axis = 0)
savename = df
print('Okay Finished')

Okay Finished


In [6]:
pl2020 = savename

In [7]:
pl2020 = pl2020.drop(columns = ['StateFIPS', 'GeoFIPS', 'GEO_ID']).reset_index(drop = True)

In [8]:
pl2020.head()

,NAME,Population,Households
0,"Montgomery County, Tennessee",220069,85714
1,"Rutherford County, Tennessee",341486,131216
2,"Sumner County, Tennessee",196281,78995
3,"Trousdale County, Tennessee",11615,3749
4,"Williamson County, Tennessee",247726,91133


In [9]:
data = pl2020.set_index('NAME').transpose()
cols = data.columns
data[cols] = data[cols].astype(float)

In [10]:
GNRCCounties = [data['Stewart County, Tennessee'],data['Montgomery County, Tennessee'],
                data['Houston County, Tennessee'],data['Humphreys County, Tennessee'],
                data['Dickson County, Tennessee'],data['Cheatham County, Tennessee'],
                data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
                data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
                data['Trousdale County, Tennessee'],data['Williamson County, Tennessee'],
                data['Rutherford County, Tennessee']]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data['Stewart County, Tennessee'],data['Montgomery County, Tennessee'],
                   data['Houston County, Tennessee'],data['Humphreys County, Tennessee'],
                   data['Dickson County, Tennessee'],data['Cheatham County, Tennessee'],
                   data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
                   data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
                   data['Trousdale County, Tennessee'],data['Williamson County, Tennessee'],
                   data['Rutherford County, Tennessee'],data['Maury County, Tennessee']]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data['Robertson County, Tennessee'],data['Sumner County, Tennessee'],
               data['Davidson County, Tennessee'],data['Wilson County, Tennessee'],
               data['Williamson County, Tennessee'],data['Rutherford County, Tennessee'],
               data['Maury County, Tennessee']]
data['MPO'] = sum(MPOCounties)
RuthInc = [data['Eagleville city, Tennessee'],data['La Vergne city, Tennessee'],
           data['Murfreesboro city, Tennessee'],data['Smyrna town, Tennessee']]
data['Rutherford Incorporated'] = sum(RuthInc)
data['Rutherford Unincorporated'] = data['Rutherford County, Tennessee'] - data['Rutherford Incorporated']
WilsonInc = [data['Lebanon city, Tennessee'],data['Mount Juliet city, Tennessee'],
             data['Watertown city, Tennessee']]
data['Wilson Incorporated'] = sum(WilsonInc)
data['Wilson Unincorporated'] = data['Wilson County, Tennessee'] - data['Wilson Incorporated']
CheathInc = [data['Ashland City town, Tennessee'],data['Kingston Springs town, Tennessee'],
             data['Pegram town, Tennessee'],data['Pleasant View city, Tennessee']]
data['Cheatham Incorporated'] = sum(CheathInc)
data['Cheatham Unincorporated'] = data['Cheatham County, Tennessee'] - data['Cheatham Incorporated']
DicksInc = [data['Burns town, Tennessee'],data['Charlotte town, Tennessee'],
            data['Dickson city, Tennessee'],data['Slayden town, Tennessee'],
            data['Vanleer town, Tennessee'],data['White Bluff town, Tennessee']]
data['Dickson Incorporated'] = sum(DicksInc)
data['Dickson Unincorporated'] = data['Dickson County, Tennessee'] - data['Dickson Incorporated']
HumphInc = [data['McEwen city, Tennessee'],data['New Johnsonville city, Tennessee'],
            data['Waverly city, Tennessee']]
data['Humphreys Incorporated'] = sum(HumphInc)
data['Humphreys Unincorporated'] = data['Humphreys County, Tennessee'] - data['Humphreys Incorporated']
data['Montgomery Incorporated'] = data['Clarksville city, Tennessee']
data['Montgomery Unincorporated'] = data['Montgomery County, Tennessee'] - data['Montgomery Incorporated']

In [11]:
pl2020 = data.transpose().reset_index()

In [12]:
#this already has the 2020 PL number for 2020 population
data = pd.read_csv('../../Data Downloads/CensusBureau_HistoricalPopulation.csv')
data = data.set_index(['NAME', 'GEO_ID'])
data = data[['1900','1910', '1920', '1930', '1940', '1950', '1960', '1970', '1980', '1990', '2000', '2010', '2020']]
data.columns = pd.MultiIndex.from_product([['Population'], data.columns, ['None']])
data.head()

Population                                                                         
                                                    1900 1910 1920 1930  1940  1950  1960  1970  1980   1990   2000   2010   2020
                                                    None None None None  None  None  None  None  None   None   None   None   None
NAME                         GEO_ID                                                                                              
Adams city, Tennessee        1600000US4700200          0  542  672    0     0     0     0   458   600    587    566    633    624
Ashland City town, Tennessee 1600000US4702180          0  641  649  712   957  1024  1400  2027  2329   2552   3641   4541   5193
Belle Meade city, Tennessee  1600000US4704620          0    0    0    0  2061  2831  3082     0  3182   2839   2943   2912   2901
Berry Hill city, Tennessee   1600000US4705140          0    0    0    0     0  1248  1551     0  1113    802    674    537   2112
Brentwood city, Tennessee    1600000US4708280          0    0    0    0     0     0     0     0  9431  16392  23445  37060  45373

In [15]:
#this already has the 2020 PL number for 2020 population
data = pd.read_csv('../../Data Downloads/CensusBureau_HistoricalPopulation.csv')
#data = data.set_index(['NAME', 'GEO_ID'])
#data = data[['1900','1910', '1920', '1930', '1940', '1950', '1960', '1970', '1980', '1990', '2000', '2010', '2020']]
#data.columns = pd.MultiIndex.from_product([['Population'], data.columns, ['None']])
data.head()

,NAME,GEO_ID,1800,1810,1820,1830,1840,1850,1860,1870,1880,1890,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010,2020
0,"Adams city, Tennessee",1600000US4700200,0,0,0,0,0,0,0,0,0,0,0,542,672,0,0,0,0,458,600,587,566,633,624
1,"Ashland City town, Tennessee",1600000US4702180,0,0,0,0,0,0,0,0,0,0,0,641,649,712,957,1024,1400,2027,2329,2552,3641,4541,5193
2,"Belle Meade city, Tennessee",1600000US4704620,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2061,2831,3082,0,3182,2839,2943,2912,2901
3,"Berry Hill city, Tennessee",1600000US4705140,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1248,1551,0,1113,802,674,537,2112
4,"Brentwood city, Tennessee",1600000US4708280,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9431,16392,23445,37060,45373


In [19]:
portlandhistoric = data.loc[data['NAME'] == 'Portland city, Tennessee']
portlandhistoric.to_csv('PortlandHistoricPopulation.csv', index = False)
portlandhistoric.head()

,NAME,GEO_ID,1800,1810,1820,1830,1840,1850,1860,1870,1880,1890,1900,1910,1920,1930,1940,1950,1960,1970,1980,1990,2000,2010,2020
48,"Portland city, Tennessee",1600000US4760280,0,0,0,0,0,0,0,0,0,0,0,579,869,1030,1212,1660,2424,2872,4030,5165,8458,11480,13156


In [14]:
test = data.reset_index(drop = False)
portland = test.loc[test['NAME'] == 'GNRC Region']
portland

NAME GEO_ID Population                                                                                                     
                             1900    1910    1920    1930    1940    1950    1960    1970     1980     1990     2000     2010     2020
                             None    None    None    None    None    None    None    None     None     None     None     None     None
72  GNRC Region    NaN     409535  428860  441256  487642  544987  617205  723279  837118  1022572  1178548  1481223  1803604  2190892

In [15]:
# data[('Population % Change', 'None','1800-1810')] = percentchange(data[('Population','1810','None')], data[('Population','1800','None')])
# data[('Population % Change', 'None','1810-1820')] = percentchange(data[('Population','1820','None')], data[('Population','1810','None')])
# data[('Population % Change', 'None','1820-1830')] = percentchange(data[('Population','1830','None')], data[('Population','1820','None')])
# data[('Population % Change', 'None','1830-1840')] = percentchange(data[('Population','1840','None')], data[('Population','1830','None')])
# data[('Population % Change', 'None','1840-1850')] = percentchange(data[('Population','1850','None')], data[('Population','1840','None')])
# data[('Population % Change', 'None','1850-1860')] = percentchange(data[('Population','1860','None')], data[('Population','1850','None')])
# data[('Population % Change', 'None','1860-1870')] = percentchange(data[('Population','1870','None')], data[('Population','1860','None')])
# data[('Population % Change', 'None','1870-1880')] = percentchange(data[('Population','1880','None')], data[('Population','1870','None')])
# data[('Population % Change', 'None','1880-1890')] = percentchange(data[('Population','1890','None')], data[('Population','1880','None')])
# data[('Population % Change', 'None','1890-1900')] = percentchange(data[('Population','1900','None')], data[('Population','1890','None')])
data[('Population % Change', 'None','1900-1910')] = percentchange(data[('Population','1910','None')], data[('Population','1900','None')])
data[('Population % Change', 'None','1910-1920')] = percentchange(data[('Population','1920','None')], data[('Population','1910','None')])
data[('Population % Change', 'None','1920-1930')] = percentchange(data[('Population','1930','None')], data[('Population','1920','None')])
data[('Population % Change', 'None','1930-1940')] = percentchange(data[('Population','1940','None')], data[('Population','1930','None')])
data[('Population % Change', 'None','1940-1950')] = percentchange(data[('Population','1950','None')], data[('Population','1940','None')])
data[('Population % Change', 'None','1950-1960')] = percentchange(data[('Population','1960','None')], data[('Population','1950','None')])
data[('Population % Change', 'None','1960-1970')] = percentchange(data[('Population','1970','None')], data[('Population','1960','None')])
data[('Population % Change', 'None','1970-1980')] = percentchange(data[('Population','1980','None')], data[('Population','1970','None')])
data[('Population % Change', 'None','1980-1990')] = percentchange(data[('Population','1990','None')], data[('Population','1980','None')])
data[('Population % Change', 'None','1990-2000')] = percentchange(data[('Population','2000','None')], data[('Population','1990','None')])
data[('Population % Change', 'None','2000-2010')] = percentchange(data[('Population','2010','None')], data[('Population','2000','None')])
data[('Population % Change', 'None','2010-2020')] = percentchange(data[('Population','2020','None')], data[('Population','2010','None')])

# data[('Population Change', 'None','1800-1810')] = realchange(data[('Population','1810','None')], data[('Population','1800','None')])
# data[('Population Change', 'None','1810-1820')] = realchange(data[('Population','1820','None')], data[('Population','1810','None')])
# data[('Population Change', 'None','1820-1830')] = realchange(data[('Population','1830','None')], data[('Population','1820','None')])
# data[('Population Change', 'None','1830-1840')] = realchange(data[('Population','1840','None')], data[('Population','1830','None')])
# data[('Population Change', 'None','1840-1850')] = realchange(data[('Population','1850','None')], data[('Population','1840','None')])
# data[('Population Change', 'None','1850-1860')] = realchange(data[('Population','1860','None')], data[('Population','1850','None')])
# data[('Population Change', 'None','1860-1870')] = realchange(data[('Population','1870','None')], data[('Population','1860','None')])
# data[('Population Change', 'None','1870-1880')] = realchange(data[('Population','1880','None')], data[('Population','1870','None')])
# data[('Population Change', 'None','1880-1890')] = realchange(data[('Population','1890','None')], data[('Population','1880','None')])
# data[('Population Change', 'None','1890-1900')] = realchange(data[('Population','1900','None')], data[('Population','1890','None')])
data[('Population Change', 'None','1900-1910')] = realchange(data[('Population','1910','None')], data[('Population','1900','None')])
data[('Population Change', 'None','1910-1920')] = realchange(data[('Population','1920','None')], data[('Population','1910','None')])
data[('Population Change', 'None','1920-1930')] = realchange(data[('Population','1930','None')], data[('Population','1920','None')])
data[('Population Change', 'None','1930-1940')] = realchange(data[('Population','1940','None')], data[('Population','1930','None')])
data[('Population Change', 'None','1940-1950')] = realchange(data[('Population','1950','None')], data[('Population','1940','None')])
data[('Population Change', 'None','1950-1960')] = realchange(data[('Population','1960','None')], data[('Population','1950','None')])
data[('Population Change', 'None','1960-1970')] = realchange(data[('Population','1970','None')], data[('Population','1960','None')])
data[('Population Change', 'None','1970-1980')] = realchange(data[('Population','1980','None')], data[('Population','1970','None')])
data[('Population Change', 'None','1980-1990')] = realchange(data[('Population','1990','None')], data[('Population','1980','None')])
data[('Population Change', 'None','1990-2000')] = realchange(data[('Population','2000','None')], data[('Population','1990','None')])
data[('Population Change', 'None','2000-2010')] = realchange(data[('Population','2010','None')], data[('Population','2000','None')])
data[('Population Change', 'None','2010-2020')] = realchange(data[('Population','2020','None')], data[('Population','2010','None')])

In [16]:
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_2':'Year', 'level_3':'Time Frame'})
historicaltidy = data

In [17]:
#"Table format"
historicalt = pd.read_csv('../../Data Downloads/CensusBureau_HistoricalPopulation.csv')
historicalt = historicalt.set_index(['NAME', 'GEO_ID'])
historicalt = historicalt[['1900','1910', '1920', '1930', '1940', '1950', '1960', '1970', '1980', '1990', '2000', '2010', '2020']]
historicalt = historicalt.add_prefix('Population ')

### Households

In [22]:
#households
data = pd.read_csv('../../Data Downloads/WoodsPooleUrbanSIM_HouseholdsProjections_2017Base.csv')

In [23]:
#merge pl2020 and replace 2020 WP # with Census #
real2020 = pl2020.drop(columns = 'Population')
data = data.merge(real2020, on = 'NAME')
data['2020'] = data['Households']
data = data.drop(columns = 'Households')

In [24]:
data.tail()

,NAME,GEO_ID,2017,2020,2025,2035,2045
78,"White Bluff town, Tennessee",1600000US4779980,1426,1597.0,1596,1725,1852
79,"White House city, Tennessee",1600000US4780200,4305,5039.0,5394,6413,7427
80,GNRC,NaN,711137,864517.0,845208,996443,1171266
81,MPO,NaN,623540,759407.0,737550,865792,1014756
82,GNRC Region,NaN,744469,907660.0,882842,1038007,1216853


In [25]:
data = data.set_index(['NAME', 'GEO_ID'])
data.columns = pd.MultiIndex.from_product([['Households Projected'], data.columns, ['None']])
cols = data.columns
data[cols] = data[cols].astype(float)
data.head(2)

Households Projected                                        
                                                          2017      2020      2025      2035      2045
                                                          None      None      None      None      None
NAME                       GEO_ID                                                                     
Cheatham County, Tennessee 0500000US47021              14671.0   16785.0   17101.0   19514.0   22019.0
Davidson County, Tennessee 0500000US47037             273497.0  328309.0  297798.0  315192.0  332379.0

In [26]:
data[('Households Projected % Change', 'None','2017-2020')] = percentchange(data[('Households Projected','2020','None')], data[('Households Projected','2017','None')])
data[('Households Projected % Change', 'None','2020-2025')] = percentchange(data[('Households Projected','2025','None')], data[('Households Projected','2020','None')])
data[('Households Projected % Change', 'None','2020-2035')] = percentchange(data[('Households Projected','2035','None')], data[('Households Projected','2020','None')])
data[('Households Projected % Change', 'None','2020-2045')] = percentchange(data[('Households Projected','2045','None')], data[('Households Projected','2020','None')])
data[('Households Projected % Change', 'None','2025-2035')] = percentchange(data[('Households Projected','2035','None')], data[('Households Projected','2025','None')])
data[('Households Projected % Change', 'None','2035-2045')] = percentchange(data[('Households Projected','2045','None')], data[('Households Projected','2035','None')])

data[('Households Projected Change', 'None','2017-2020')] = realchange(data[('Households Projected','2020','None')], data[('Households Projected','2017','None')])
data[('Households Projected Change', 'None','2020-2025')] = realchange(data[('Households Projected','2025','None')], data[('Households Projected','2020','None')])
data[('Households Projected Change', 'None','2020-2035')] = realchange(data[('Households Projected','2035','None')], data[('Households Projected','2020','None')])
data[('Households Projected Change', 'None','2020-2045')] = realchange(data[('Households Projected','2045','None')], data[('Households Projected','2020','None')])
data[('Households Projected Change', 'None','2025-2035')] = realchange(data[('Households Projected','2035','None')], data[('Households Projected','2025','None')])
data[('Households Projected Change', 'None','2035-2045')] = realchange(data[('Households Projected','2045','None')], data[('Households Projected','2035','None')])

In [27]:
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_2':'Year', 'level_3':'Time Frame'})
#data.tail()
hhstidy = data

In [28]:
#"Table format"
hhst = pd.read_csv('../../Data Downloads/WoodsPooleUrbanSIM_HouseholdsProjections_2017Base.csv')

In [29]:
#merge pl2020 and replace 2020 WP # with Census #
real2020 = pl2020.drop(columns = 'Population')
hhst = hhst.merge(real2020, on = 'NAME')
hhst['2020'] = hhst['Households']
hhst = hhst.drop(columns = 'Households')

In [30]:
hhst = hhst.set_index(['NAME', 'GEO_ID'])
hhst = hhst.add_prefix('Projected Households ')
hhst.head()

,,Projected Households 2017,Projected Households 2020,Projected Households 2025,Projected Households 2035,Projected Households 2045
NAME,GEO_ID,,,,,
"Cheatham County, Tennessee",0500000US47021,14671,16785.0,17101,19514,22019
"Davidson County, Tennessee",0500000US47037,273497,328309.0,297798,315192,332379
"Dickson County, Tennessee",1600000US4720620,19032,22551.0,21596,23868,26141
"Houston County, Tennessee",0500000US47083,2999,3936.0,3444,3840,4230
"Humphreys County, Tennessee",0500000US47085,7064,8849.0,7622,7949,8247


In [31]:
#populations
data = pd.read_csv('../../Data Downloads/WoodsPooleUrbanSIM_PopulationProjections_2017Base.csv')
data.head()

,NAME,GEO_ID,2017,2020,2025,2035,2045
0,"Cheatham County, Tennessee",0500000US47021,38688,41240,45070,51373,58015
1,"Davidson County, Tennessee",0500000US47037,638443,663737,695114,735884,776083
2,"Dickson County, Tennessee",1600000US4720620,49003,51810,55652,61355,67195
3,"Houston County, Tennessee",0500000US47083,7522,7958,8596,9631,10608
4,"Humphreys County, Tennessee",0500000US47085,17192,17802,18499,19284,19998


In [32]:
test = data.reset_index(drop = False)
portland = test.loc[test['NAME'] == 'GNRC Region']
portland

,index,NAME,GEO_ID,2017,2020,2025,2035,2045
95,95,GNRC Region,NaN,1903165,2047709,2268607,2683897,3164912


In [33]:
#merge pl2020 and replace 2020 WP # with Census #
real2020 = pl2020.drop(columns = 'Households')
data = data.merge(real2020, on = 'NAME')
data['2020'] = data['Population']
data = data.drop(columns = 'Population')

In [34]:
data.head()

,NAME,GEO_ID,2017,2020,2025,2035,2045
0,"Cheatham County, Tennessee",0500000US47021,38688,41072.0,45070,51373,58015
1,"Davidson County, Tennessee",0500000US47037,638443,715884.0,695114,735884,776083
2,"Dickson County, Tennessee",1600000US4720620,49003,54315.0,55652,61355,67195
3,"Houston County, Tennessee",0500000US47083,7522,8283.0,8596,9631,10608
4,"Humphreys County, Tennessee",0500000US47085,17192,18990.0,18499,19284,19998


In [35]:
data = data.set_index(['NAME', 'GEO_ID'])
data.columns = pd.MultiIndex.from_product([['Population Projected'], data.columns, ['None']])
cols = data.columns
data[cols] = data[cols].astype(float)

In [36]:
data[('Population Projected % Change', 'None','2017-2020')] = percentchange(data[('Population Projected','2020','None')], data[('Population Projected','2017','None')])
data[('Population Projected % Change', 'None','2020-2025')] = percentchange(data[('Population Projected','2025','None')], data[('Population Projected','2020','None')])
data[('Population Projected % Change', 'None','2020-2035')] = percentchange(data[('Population Projected','2035','None')], data[('Population Projected','2020','None')])
data[('Population Projected % Change', 'None','2020-2045')] = percentchange(data[('Population Projected','2045','None')], data[('Population Projected','2020','None')])
data[('Population Projected % Change', 'None','2025-2035')] = percentchange(data[('Population Projected','2035','None')], data[('Population Projected','2025','None')])
data[('Population Projected % Change', 'None','2035-2045')] = percentchange(data[('Population Projected','2045','None')], data[('Population Projected','2035','None')])

data[('Population Projected Change', 'None','2017-2020')] = realchange(data[('Population Projected','2020','None')], data[('Population Projected','2017','None')])
data[('Population Projected Change', 'None','2020-2025')] = realchange(data[('Population Projected','2025','None')], data[('Population Projected','2020','None')])
data[('Population Projected Change', 'None','2020-2035')] = realchange(data[('Population Projected','2035','None')], data[('Population Projected','2020','None')])
data[('Population Projected Change', 'None','2020-2045')] = realchange(data[('Population Projected','2045','None')], data[('Population Projected','2020','None')])
data[('Population Projected Change', 'None','2025-2035')] = realchange(data[('Population Projected','2035','None')], data[('Population Projected','2025','None')])
data[('Population Projected Change', 'None','2035-2045')] = realchange(data[('Population Projected','2045','None')], data[('Population Projected','2035','None')])

In [37]:
data = data.stack([1, 1])
data = data.reset_index(drop = False)
data = data.rename(columns = {'level_2':'Year', 'level_3':'Time Frame'})
#data.tail()
poptidy = data

In [38]:
#"Table format"
popt = pd.read_csv('../../Data Downloads/WoodsPooleUrbanSIM_PopulationProjections_2017Base.csv')

In [39]:
#merge pl2020 and replace 2020 WP # with Census #
real2020 = pl2020.drop(columns = 'Households')
popt = popt.merge(real2020, on = 'NAME')
popt['2020'] = popt['Population']
popt = popt.drop(columns = 'Population')

In [40]:
popt = popt.set_index(['NAME', 'GEO_ID'])
popt = popt.add_prefix('Projected Population ')
popt.head()

,,Projected Population 2017,Projected Population 2020,Projected Population 2025,Projected Population 2035,Projected Population 2045
NAME,GEO_ID,,,,,
"Cheatham County, Tennessee",0500000US47021,38688,41072.0,45070,51373,58015
"Davidson County, Tennessee",0500000US47037,638443,715884.0,695114,735884,776083
"Dickson County, Tennessee",1600000US4720620,49003,54315.0,55652,61355,67195
"Houston County, Tennessee",0500000US47083,7522,8283.0,8596,9631,10608
"Humphreys County, Tennessee",0500000US47085,17192,18990.0,18499,19284,19998


# Join all

In [41]:
dfs = [historicaltidy, poptidy, hhstidy]

In [42]:
datatidy = reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'GEO_ID', 'Year', 'Time Frame'],
                                           how='outer'), dfs)

In [43]:
datatidy = datatidy.set_index('NAME').transpose()

In [44]:
datatidy = datatidy.rename(columns = {'Adams city, Tennessee': 'Adams',
 'Ashland City town, Tennessee': 'Ashland City',
 'Belle Meade city, Tennessee': 'Belle Meade',
 'Berry Hill city, Tennessee': 'Berry Hill',
 'Brentwood city, Tennessee': 'Brentwood',
 'Burns town, Tennessee': 'Burns',
 'Cedar Hill city, Tennessee': 'Cedar Hill',
 'Charlotte town, Tennessee': 'Charlotte',
 'Cheatham County, Tennessee': 'Cheatham County',
 'Clarksville city, Tennessee': 'Clarksville',
 'Columbia city, Tennessee': 'Columbia',
 'Coopertown town, Tennessee': 'Coopertown',
 'Cross Plains city, Tennessee': 'Cross Plains',
 'Cumberland City town, Tennessee': 'Cumberland City',
 'Davidson County, Tennessee': 'Davidson County',
 'Dickson city, Tennessee': 'Dickson',
 'Dickson County, Tennessee': 'Dickson County',
 'Dover city, Tennessee': 'Dover',
 'Eagleville city, Tennessee': 'Eagleville',
 'Erin city, Tennessee': 'Erin',
 'Fairview city, Tennessee': 'Fairview',
 'Forest Hills city, Tennessee': 'Forest Hills',
 'Franklin city, Tennessee': 'Franklin',
 'Gallatin city, Tennessee': 'Gallatin',
 'Goodlettsville city, Tennessee': 'Goodlettsville',
 'Greenbrier town, Tennessee': 'Greenbrier',
 'Hartsville/Trousdale County, Tennessee': 'Hartsville/Trousdale',
 'Hendersonville city, Tennessee': 'Hendersonville',
 'Houston County, Tennessee': 'Houston County',
 'Humphreys County, Tennessee': 'Humphreys County',
 'Kingston Springs town, Tennessee': 'Kingston Springs',
 'La Vergne city, Tennessee': 'La Vergne',
 'Lebanon city, Tennessee': 'Lebanon',
 'Maury County, Tennessee': 'Maury County',
 'McEwen city, Tennessee': 'McEwen',
 'Millersville city, Tennessee': 'Millersville',
 'Mitchellville city, Tennessee': 'Mitchellville',
 'Montgomery County, Tennessee': 'Montgomery County',
 'Mount Juliet city, Tennessee': 'Mount Juliet',
 'Mount Pleasant city, Tennessee': 'Mount Pleasant',
 'Murfreesboro city, Tennessee': 'Murfreesboro',
 'Nashville-Davidson metropolitan government (balance), Tennessee': 'Nashville',
 'New Johnsonville city, Tennessee': 'New Johnsonville',
 'Nolensville town, Tennessee': 'Nolensville',
 'Oak Hill city, Tennessee': 'Oak Hill',
 'Pegram town, Tennessee': 'Pegram',
 'Pleasant View city, Tennessee': 'Pleasant View',
 'Portland city, Tennessee': 'Portland',
 'Ridgetop city, Tennessee': 'Ridgetop',
 'Robertson County, Tennessee': 'Robertson County',
 'Rutherford County, Tennessee': 'Rutherford County',
 'Slayden town, Tennessee': 'Slayden',
 'Smyrna town, Tennessee': 'Smyrna',
 'Spring Hill city, Tennessee': 'Spring Hill',
 'Springfield city, Tennessee': 'Springfield',
 'Stewart County, Tennessee': 'Stewart County',
 'Sumner County, Tennessee': 'Sumner County',
 'Tennessee Ridge town, Tennessee': 'Tennessee Ridge',
 "Thompson's Station town, Tennessee": "Thompson's Station",
 'Trousdale County, Tennessee': 'Trousdale County',
 'Vanleer town, Tennessee': 'Vanleer',
 'Watertown city, Tennessee': 'Watertown',
 'Waverly city, Tennessee': 'Waverly',
 'Westmoreland town, Tennessee': 'Westmoreland',
 'White Bluff town, Tennessee': 'White Bluff',
 'White House city, Tennessee': 'White House',
 'Williamson County, Tennessee': 'Williamson County',
 'Wilson County, Tennessee': 'Wilson County'})

In [45]:
datatidy = datatidy.transpose()
datatidy = datatidy.reset_index()
datatidy.head()

,NAME,GEO_ID,Year,Time Frame,Population,Population % Change,Population Change,Population Projected,Population Projected % Change,Population Projected Change,Households Projected,Households Projected % Change,Households Projected Change
0,Adams,1600000US4700200,1900,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Adams,1600000US4700200,1910,None,542.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Adams,1600000US4700200,1920,None,672.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Adams,1600000US4700200,1930,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Adams,1600000US4700200,1940,None,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
datatidy.to_feather('../../Outputs/HIST_PROJECT_PopulationTIDY')

In [47]:
dfs = [historicalt, popt, hhst]
datat = reduce(lambda  left,right: pd.merge(left,right, on = ['NAME', 'GEO_ID'],
                                            how='outer'), dfs)

In [48]:
datat = datat.reset_index()

In [49]:
datat = datat.set_index('NAME').transpose()

In [50]:
datat = datat.rename(columns = {'Adams city, Tennessee': 'Adams',
 'Ashland City town, Tennessee': 'Ashland City',
 'Belle Meade city, Tennessee': 'Belle Meade',
 'Berry Hill city, Tennessee': 'Berry Hill',
 'Brentwood city, Tennessee': 'Brentwood',
 'Burns town, Tennessee': 'Burns',
 'Cedar Hill city, Tennessee': 'Cedar Hill',
 'Charlotte town, Tennessee': 'Charlotte',
 'Cheatham County, Tennessee': 'Cheatham County',
 'Clarksville city, Tennessee': 'Clarksville',
 'Columbia city, Tennessee': 'Columbia',
 'Coopertown town, Tennessee': 'Coopertown',
 'Cross Plains city, Tennessee': 'Cross Plains',
 'Cumberland City town, Tennessee': 'Cumberland City',
 'Davidson County, Tennessee': 'Davidson County',
 'Dickson city, Tennessee': 'Dickson',
 'Dickson County, Tennessee': 'Dickson County',
 'Dover city, Tennessee': 'Dover',
 'Eagleville city, Tennessee': 'Eagleville',
 'Erin city, Tennessee': 'Erin',
 'Fairview city, Tennessee': 'Fairview',
 'Forest Hills city, Tennessee': 'Forest Hills',
 'Franklin city, Tennessee': 'Franklin',
 'Gallatin city, Tennessee': 'Gallatin',
 'Goodlettsville city, Tennessee': 'Goodlettsville',
 'Greenbrier town, Tennessee': 'Greenbrier',
 'Hartsville/Trousdale County, Tennessee': 'Hartsville/Trousdale',
 'Hendersonville city, Tennessee': 'Hendersonville',
 'Houston County, Tennessee': 'Houston County',
 'Humphreys County, Tennessee': 'Humphreys County',
 'Kingston Springs town, Tennessee': 'Kingston Springs',
 'La Vergne city, Tennessee': 'La Vergne',
 'Lebanon city, Tennessee': 'Lebanon',
 'Maury County, Tennessee': 'Maury County',
 'McEwen city, Tennessee': 'McEwen',
 'Millersville city, Tennessee': 'Millersville',
 'Mitchellville city, Tennessee': 'Mitchellville',
 'Montgomery County, Tennessee': 'Montgomery County',
 'Mount Juliet city, Tennessee': 'Mount Juliet',
 'Mount Pleasant city, Tennessee': 'Mount Pleasant',
 'Murfreesboro city, Tennessee': 'Murfreesboro',
 'Nashville-Davidson metropolitan government (balance), Tennessee': 'Nashville',
 'New Johnsonville city, Tennessee': 'New Johnsonville',
 'Nolensville town, Tennessee': 'Nolensville',
 'Oak Hill city, Tennessee': 'Oak Hill',
 'Pegram town, Tennessee': 'Pegram',
 'Pleasant View city, Tennessee': 'Pleasant View',
 'Portland city, Tennessee': 'Portland',
 'Ridgetop city, Tennessee': 'Ridgetop',
 'Robertson County, Tennessee': 'Robertson County',
 'Rutherford County, Tennessee': 'Rutherford County',
 'Slayden town, Tennessee': 'Slayden',
 'Smyrna town, Tennessee': 'Smyrna',
 'Spring Hill city, Tennessee': 'Spring Hill',
 'Springfield city, Tennessee': 'Springfield',
 'Stewart County, Tennessee': 'Stewart County',
 'Sumner County, Tennessee': 'Sumner County',
 'Tennessee Ridge town, Tennessee': 'Tennessee Ridge',
 "Thompson's Station town, Tennessee": "Thompson's Station",
 'Trousdale County, Tennessee': 'Trousdale County',
 'Vanleer town, Tennessee': 'Vanleer',
 'Watertown city, Tennessee': 'Watertown',
 'Waverly city, Tennessee': 'Waverly',
 'Westmoreland town, Tennessee': 'Westmoreland',
 'White Bluff town, Tennessee': 'White Bluff',
 'White House city, Tennessee': 'White House',
 'Williamson County, Tennessee': 'Williamson County',
 'Wilson County, Tennessee': 'Wilson County'})

In [51]:
datat = datat.transpose()
datat = datat.reset_index()
datat.head()

,NAME,GEO_ID,Population 1900,Population 1910,Population 1920,Population 1930,Population 1940,Population 1950,Population 1960,Population 1970,Population 1980,Population 1990,Population 2000,Population 2010,Population 2020,Projected Population 2017,Projected Population 2020,Projected Population 2025,Projected Population 2035,Projected Population 2045,Projected Households 2017,Projected Households 2020,Projected Households 2025,Projected Households 2035,Projected Households 2045
0,Adams,1600000US4700200,0.0,542.0,672.0,0.0,0.0,0.0,0.0,458.0,600.0,587.0,566.0,633.0,624.0,728.0,624.0,801.0,876.0,995.0,228.0,250.0,249.0,274.0,309.0
1,Ashland City,1600000US4702180,0.0,641.0,649.0,712.0,957.0,1024.0,1400.0,2027.0,2329.0,2552.0,3641.0,4541.0,5193.0,4701.0,5193.0,6315.0,7850.0,9431.0,2070.0,2311.0,2737.0,3428.0,4101.0
2,Belle Meade,1600000US4704620,0.0,0.0,0.0,0.0,2061.0,2831.0,3082.0,0.0,3182.0,2839.0,2943.0,2912.0,2901.0,2615.0,2901.0,2790.0,2875.0,2959.0,1021.0,1130.0,1081.0,1114.0,1146.0
3,Berry Hill,1600000US4705140,0.0,0.0,0.0,0.0,0.0,1248.0,1551.0,0.0,1113.0,802.0,674.0,537.0,2112.0,886.0,2112.0,1280.0,1702.0,2242.0,536.0,1657.0,726.0,913.0,1171.0
4,Brentwood,1600000US4708280,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9431.0,16392.0,23445.0,37060.0,45373.0,39849.0,45373.0,43876.0,48343.0,52655.0,13277.0,15279.0,14596.0,16051.0,17507.0


In [52]:
datat.to_feather('../../Outputs/HIST_PROJECT_PopulationSUFFIX')